# Simple parking lot test
We train the network on mock satellite images of a small surface lot. The computer-generated images are very simple colored drawings. This way we can produce a lot of training-test samples.

In the first test, there will be **10 spaces**, with a random number of cars occupying them. The goal of the network is to highlight which spaces are open or not.

# Import libraries

In [ ]:
import numpy as np
import math 
import random 
from plotly.offline import init_notebook_mode
init_notebook_mode()
%matplotlib inline
import matplotlib.pyplot as plt

from keras import models
from keras import layers
from keras.models import load_model
from keras import initializers
from keras import optimizers
from keras import regularizers
from keras import backend as K
from sklearn.model_selection import train_test_split

# Object dimensions and colors

These are just made-up parameters for now. I don't have the exact proportionalities figured out yet. Need to do some research.

In [ ]:
# parking lot size
lot_height = 51
lot_width = 71

# parking lot box 
box_height = 31
box_length = 50
line_width = 1
space_width = 10

spaces_per_row = 5
number_of_groups = 1

# car size
car_length = 12
car_width = 7

# number of cars
max_cars = 2 * spaces_per_row * number_of_groups

# randomly displace cars
displace_car = True

# add gaussian noise to color
gaussian_noise = True

# random people (noise)
add_people = True
max_people = 10

# amount of data   
tagged_samples = 100000
test_samples = 1000


# Make the background

The pavement will just be gray for now (images are grayscale)

In [ ]:
background = 100 * np.ones(lot_height * lot_width)
background = background.reshape(lot_height, lot_width)

plt.imshow(background, cmap = 'gray', vmin = 0, vmax = 255)
plt.show()


# Make the parking lot box 

In [ ]:
def add_parking_lot_box(image, box_height, box_length, line_width, shift, space_width, spaces_per_row):

    # add white vertical lines
    for n in range(spaces_per_row + 1):
        for i in range(box_height):
            image[i + shift][(space_width*(n+1)):(space_width*(n+1) + line_width)] = 255    
            
    # add white horizontal lines
    mid_point = shift + math.floor(box_height/2)
    for i in range(line_width):
        image[mid_point + i][space_width:(space_width + box_length)] = 255  

    return image

shift1 = space_width

parking_lot = background.copy()
parking_lot = add_parking_lot_box(parking_lot, box_height, box_length, line_width, shift1, space_width, spaces_per_row)

plt.imshow(parking_lot, cmap = 'gray', vmin = 0, vmax = 255)
plt.show()

# Make one noisy background

For training and validation

In [ ]:
parking_lot_noise = parking_lot.copy()

for i in range(parking_lot_noise.shape[0]):
    for j in range(parking_lot_noise .shape[1]):
        noise = max(-6, min(6, np.random.normal(0,3)))
        parking_lot_noise [i][j] = max(0, min(255, parking_lot_noise [i][j] + noise))
        
plt.imshow(parking_lot_noise, cmap = 'gray', vmin = 0, vmax = 255)
plt.show()

# Add one car somewhere

**The cars have 12 possible colors** (not including background color = 100)

**Option to displace car slightly**

In [ ]:
def add_car(parking_lot, displace_car, i0, j0, car_length, car_width):
    # 12 possible colors =/= background color 
    # (0, 10, 20, 30, 40, 50, 60, 140, 150, 160, 170, 180, 190, 200)
    color1 = 10 * random.randint(0, 6)
    color2 = 10 * random.randint(14, 20)
    color = np.random.choice([color1, color2], p = [0.5, 0.5])
    
    if displace_car:
        i0 += np.random.choice([-1,0,1], p = [0.2, 0.6, 0.2])
        j0 += np.random.choice([-1,0,1], p = [0.2, 0.6, 0.2])

    for i in range(car_length):
        for j in range(car_width):
            noise = 0
            if gaussian_noise:
                noise = max(-40, min(40,np.random.normal(0,20)))
                
            parking_lot[i0 + i][j0 + j] = max(0, min(255, color + noise))
            
    return parking_lot

# starting indices in the image
i0 = space_width + line_width + 1
j0 = space_width + 2*line_width

parking_lot_fill = add_car(parking_lot.copy(), displace_car, i0 + 0*15, j0 + 3*10, car_length, car_width)
plt.imshow(parking_lot_fill, cmap = 'gray', vmin = 0, vmax = 255)
plt.show()

# Randomly filled parking lot

Randomly occupied parking lot
**Let's also add people for fun**

In [ ]:
def random_lot_2(lot, displace_car, add_people, max_people, max_cars, total_cars, spaces_per_row, i0, j0, car_length, car_width): 
    if gaussian_noise:
        for i in range(lot.shape[0]):
            for j in range(lot.shape[1]):
                noise = max(-6, min(6, np.random.normal(0,3)))
                lot[i][j] = max(0, min(255, lot[i][j] + noise))
    
    if add_people:
        people = random.randint(0, max_people)
        for n in range(people):
            i = random.randint(0, lot.shape[0] - 2)
            j = random.randint(0, lot.shape[1] - 2)
            for x in range(2):
                for y in range(2):
                    lot[i+x][j+y] = 40
    
    occupancy_index = random.sample(range(0, max_cars), total_cars)
    #occupancy = np.zeros(max_cars)
    occupancy = np.ones(max_cars)
    
    for n in range(max_cars):
        if n in occupancy_index:
            #occupancy[n] = 1
            occupancy[n] = 0
            i = math.floor(n / spaces_per_row)
            j = n  -  spaces_per_row * i
            lot = add_car(lot, displace_car, i0 + i*15, j0 + j*10, car_length, car_width)
        
    return lot, occupancy

# Tagged parking lot dataset to train network

**The tagged squares can have four different colors**

In [ ]:
# tag parking spaces
def tag_spaces(image, i0, j0):
    # random white or black tags
    color = np.random.choice([0, 40, 160, 200], p = [0.25, 0.25, 0.25, 0.25])
    for i in range(4):
        for j in range(3):
            image[i0 + i][j0 + j] = color
    return image

def tag_parking_lot(lot, max_cars, total_cars, spaces_per_row, i0, j0):
    
    occupancy_index = random.sample(range(0, max_cars), total_cars)
    #occupancy = np.zeros(max_cars)
    occupancy = np.ones(max_cars)
    
    for n in range(max_cars):
        if n in occupancy_index:
            #occupancy[n] = 1
            occupancy[n] = 0
            i = math.floor(n / spaces_per_row)
            j = n  -  spaces_per_row * i
            lot = tag_spaces(lot, i0+4 + i*15, j0+2 + j*10)
    return lot, occupancy


# tagged pretraining images
lot_tag_total = np.zeros(tagged_samples * lot_height * lot_width).reshape(tagged_samples, lot_height * lot_width)
occupancy_tag_label = np.zeros(tagged_samples * max_cars).reshape(tagged_samples, max_cars)

# random tagged parking lots
for i in range(tagged_samples):
    total_cars = random.randint(0, max_cars)
    lot, occupancy = tag_parking_lot(parking_lot_noise.copy(), max_cars, total_cars, spaces_per_row, i0, j0)
 
    # renormalize the gray-scale images
    lot_tag_total[i] = lot.reshape(lot_height * lot_width) / 255
    occupancy_tag_label[i] = occupancy


# plot a random parking lot sample
print(occupancy_tag_label[0].reshape(2,5))
plt.imshow(lot_tag_total[0].reshape(lot_height, lot_width), cmap = 'gray', vmin = 0, vmax = 1)
plt.show()

# Train a FCN network on tagged spaces

In [ ]:
train_network = True
load_network = False

hidden_nodes = 12 * max_cars
output_nodes = max_cars

hidden_activation = 'relu'
output_activation = 'sigmoid'

optimizer = 'sgd'
loss = 'binary_crossentropy'

epochs = 100
batch_size = 100

network = models.Sequential()
network.add(layers.Dense(hidden_nodes, activation = hidden_activation, input_shape = (lot_height * lot_width,)))
network.add(layers.Dense(output_nodes, activation = output_activation))
network.compile(optimizer = optimizer, loss = loss, metrics = ['accuracy'])

if load_network:
    network.load_weights('FCN_network_tagged_small.h5')

# training data = tagged parking lots
train_images, val_images, train_labels, val_labels = train_test_split(lot_tag_total, occupancy_tag_label, test_size=0.2,random_state=42)

if train_network:
    history = network.fit(train_images, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(val_images, val_labels))
    network.save('FCN_network_tagged_small.h5')
    
    

# Highlight occupied spaces to visualize predictions

In [ ]:
def highlight_occupied_spaces(image, i0, j0):
    for i in range(5):
        for j in range(5):
            color = 0
            if i in range(1,4) and j in range(1,4):
                color = 255
            image[i0 + i][j0 + j] = color  
    return image

def highlight_parking_lot(image, occupancy_predict, max_cars, spaces_per_row, i0, j0):  
    for s in range(max_cars):
        if occupancy_predict[s] == 1:
            i = math.floor(s / spaces_per_row)
            j = s  -  spaces_per_row * i
            image = highlight_occupied_spaces(image, i0+3 + i*16, j0+1 + j*10)            
    return image

# Test how well the network labels occupancy 

In [ ]:
lot_test_images = np.zeros(test_samples * lot_height * lot_width).reshape(test_samples, lot_height * lot_width)
occupancy_test_label = np.zeros(test_samples * max_cars).reshape(test_samples, max_cars)

test_accuracy_cars = np.zeros(max_cars + 1)

for total_cars in range(max_cars + 1):
    for i in range(test_samples):
        lot, occupancy = random_lot_2(parking_lot.copy(), displace_car, add_people, max_people, max_cars, total_cars, spaces_per_row, i0, j0, car_length, car_width)
    
        # renormalize the gray-scale images
        lot_test_images[i] = lot.reshape(lot_height * lot_width) / 255
        occupancy_test_label[i] = occupancy
        
    test_loss, test_accuracy = network.evaluate(lot_test_images, occupancy_test_label)
    predict_occupancy = network.predict(lot_test_images).round(0)
    
    print()
    print("Network performance:", total_cars, "car(s)")
    print("-------------------------------------")
    print("Test loss = ", test_loss)
    print("Test accuracy = ", test_accuracy)
    print()

    test_accuracy_cars[total_cars] = test_accuracy
    
    # print visual predictions
    for i in range(5):
        # undo normalization for plotting 
        unscaled_lot = (lot_test_images[i] * 255).reshape(lot_height, lot_width)
        occupancy = predict_occupancy[i]
        highlighted_parking_lot = highlight_parking_lot(unscaled_lot, occupancy, max_cars, spaces_per_row, i0, j0)

        plt.imshow(highlighted_parking_lot.reshape(lot_height, lot_width), cmap = 'gray', vmin = 0, vmax = 255)
        plt.show()

print(test_accuracy_cars)

# Plot accuracy vs number of cars

In [ ]:
import plotly.offline as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from matplotlib.pyplot import figure

In [ ]:
test_accuracy_cars.mean()

In [ ]:
bar_width = 0.9
opacity = 0.75

fig = plt.figure(figsize=(10,6))
plt.bar(np.arange(max_cars+1), test_accuracy_cars, bar_width, align='center', alpha=opacity, color='red')
plt.ylim(0.95,1.005)
plt.xticks(np.arange(0, 11, step=1))
plt.ylabel('Test accuracy')
plt.xlabel('Number of parked cars')
plt.title('FCN Performance')
plt.rc('font', size=12)          # controls default text sizes
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=16)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
plt.rc('legend', fontsize=12)    # legend fontsize
plt.rc('figure', titlesize=12)   # fontsize of the figure title
plt.show()